In [241]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Meghna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [242]:
import collections
df = pd.read_csv('/Users/Meghna/Desktop/human_annotated_black_tweets.csv')
dff = pd.read_csv('/Users/Meghna/Desktop/human_annotated_black_tweets2.csv')
dfff = pd.read_csv('/Users/Meghna/Desktop/human_annotated_nonblack_tweets.csv')

In [243]:
races = df['race'].to_list() + dff['race'].to_list() + dfff['race'].to_list()
tweets = df['tweet'].to_list() + dff['tweet'].to_list() + dfff['tweet'].to_list()

In [244]:
train_corpus,test_corpus,train_labels,test_labels = train_test_split(tweets,races,stratify=races,test_size=0.25,random_state=1)

In [245]:
# races_for_fasttext = []
# for race in new_races:
#     if race == str(0):
#         races_for_fasttext.append('__label__others')
#     else:
#         races_for_fasttext.append('__label__aa')

In [246]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+' #remove mentions
pat2 = r'https?://[^ ]+' #remove hyperlinks
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml') # remove html tags
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?") #remove byte order marks
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    lower_case = emoji.demojize(lower_case, language='en')
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) #remove hashtag
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [247]:
cleaned_tweets = []
for tweet in train_corpus:
    cleaned_tweets.append(tweet_cleaner_updated(tweet))

In [49]:
uncleaned_tweets_test = test_corpus

In [249]:
# use it when using data augmentation - EDA
with open('/Users/Meghna/Desktop/train_for_eda_human_annotated.txt','a') as file:
    for i in range(len(cleaned_tweets)):
        line = str(train_labels[i]) + '\t' + cleaned_tweets[i]
        file.write(line)
        file.write('\n')

In [251]:
# use it when using data augmentation - EDA
new_races = []
train_tweets = []

with open(r'/Users/Meghna/Desktop/eda_train_for_eda_human_annotated.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        race, tweet = line.split('\t')
        new_races.append(race)
        train_tweets.append(tweet.rstrip())

In [252]:
cleaned_tweets_test = []
for tweet in test_corpus:
    cleaned_tweets_test.append(tweet_cleaner_updated(tweet))

In [51]:
def tweet_cleaner_updated2(text):
    soup = BeautifulSoup(text, 'lxml') # remove html tags
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?") #remove byte order marks
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    return stripped

In [52]:
uncleaned_tweets = []
for tweet in train_corpus:
    uncleaned_tweets.append(tweet_cleaner_updated2(tweet))

In [53]:
uncleaned_tweets_test = []
for tweet in test_corpus:
    uncleaned_tweets_test.append(tweet_cleaner_updated2(tweet))

In [54]:
from advertools.emoji import extract_emoji
uncleaned_count_emoji = []
for tweet in uncleaned_tweets:
    count = 0
    for i in range(len(extract_emoji(tweet)['emoji_flat_text'])):
        count += 1
    uncleaned_count_emoji.append(count)

In [64]:
uncleaned_count_emoji_test = []
for tweet in uncleaned_tweets_test:
    count = 0
    for i in range(len(extract_emoji(tweet)['emoji_flat_text'])):
        count += 1
    uncleaned_count_emoji_test.append(count)

In [55]:
count_punctuation = lambda l1,l2: sum([1 for x in l1 if x in l2])
uncleaned_count_puncts = []
for tweet in uncleaned_tweets:
    uncleaned_count_puncts.append(count_punctuation(tweet,set(string.punctuation)))

In [65]:
uncleaned_count_puncts_test = []
for tweet in uncleaned_tweets_test:
    uncleaned_count_puncts_test.append(count_punctuation(tweet,set(string.punctuation)))

In [253]:
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}

code_pos_map = {v: k for k, v in  pos_code_map.items()}

In [254]:
#abbrivation converters
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [255]:
#POS tag converting
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [59]:
pos_for_tweets = []
for tweet in cleaned_tweets:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets.append(text_pos(tweet))

In [257]:
# use it when using data augmentation - EDA
pos_for_tweets = []
for tweet in train_tweets:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets.append(text_pos(tweet))

In [258]:
pos_for_tweets_test = []
for tweet in cleaned_tweets_test:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets_test.append(text_pos(tweet))

In [259]:
from sklearn.feature_extraction.text import CountVectorizer
#v = CountVectorizer(analyzer='char',ngram_range=(1, 3))

v = TfidfVectorizer(stop_words = 'english', analyzer='char',ngram_range=(1, 1))
ngrams = v.fit_transform(pos_for_tweets)
ngrams_val = v.transform(pos_for_tweets_test)

In [261]:
t = TfidfVectorizer(stop_words = 'english',max_features=10000,ngram_range=(1, 1))
tfidf_train = t.fit_transform(cleaned_tweets)


# use it when using data augmentation - EDA
#tfidf_train = t.fit_transform(train_tweets)

tfidf_val = t.transform(cleaned_tweets_test)




In [63]:
print(type(tfidf_train))

<class 'scipy.sparse.csr.csr_matrix'>


In [66]:
from scipy import sparse

In [67]:
uncleaned_count_emoji_arr = np.array(uncleaned_count_emoji)
uncleaned_count_emoji_test_arr = np.array(uncleaned_count_emoji_test)
uncleaned_count_puncts_arr = np.array(uncleaned_count_puncts)
uncleaned_count_puncts_test_arr = np.array(uncleaned_count_puncts_test)

In [94]:
uncleaned_count_emoji_arr = uncleaned_count_emoji_arr.reshape(5116,1)

In [95]:
uncleaned_count_puncts_arr = uncleaned_count_puncts_arr.reshape(5116,1)

In [96]:
uncleaned_count_emoji_test_arr = uncleaned_count_emoji_test_arr.reshape(1706,1)
uncleaned_count_puncts_test_arr = uncleaned_count_puncts_test_arr.reshape(1706,1)

In [97]:
uncleaned_count_emoji_matrix = sparse.csr_matrix(uncleaned_count_emoji_arr)
uncleaned_count_emoji_test_matrix = sparse.csr_matrix(uncleaned_count_emoji_test_arr)
uncleaned_count_puncts_matrix = sparse.csr_matrix(uncleaned_count_puncts_arr)
uncleaned_count_puncts_test_matrix = sparse.csr_matrix(uncleaned_count_puncts_test_arr)

In [98]:
tfidf_train.shape

(5116, 10000)

In [99]:
uncleaned_count_emoji_matrix.shape

(5116, 1)

In [262]:
from scipy.sparse import hstack
train_train = hstack((tfidf_train,ngrams))
test_test = hstack((tfidf_val,ngrams_val))

In [266]:
new_racess= []
for race in new_races:
    new_racess.append(int(race))
    

In [268]:
from sklearn.svm import SVC

#clf2 = SVC(random_state=0).fit(train_train, train_labels)

# use it when using data augmentation - EDA
clf2 = SVC(random_state=0).fit(train_train, new_racess)

y_pred = clf2.predict(test_test)
accuracy = accuracy_score(test_labels, y_pred)
accuracy

0.6424384525205158

In [145]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

# train_array = train_train.toarray()
# test_array = test_test.toarray()


x_train1 = pd.Series(cleaned_tweets)
x_val1 = pd.Series(cleaned_tweets_test)
# x_test_s1 = pd.Series(x_test)
# x_test_s2 = pd.Series(x_test2)
indexes = [i for i in range(5116)]
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(indexes, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result
all_x = pd.concat([x_train1,x_val1])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [146]:
len(all_x_w2v)

5116

In [147]:
model_ug_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

2021-08-15 13:59:01,013 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,s0.001,t3)', 'datetime': '2021-08-15T13:59:01.012928', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 5116/5116 [00:00<00:00, 2616836.50it/s]
2021-08-15 13:59:01,023 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 13:59:01,024 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 13:59:01,082 | INFO | doc2vec.py:1040 | scan_vocab | collected 11564 word types and 5116 unique tags from a corpus of 5116 examples and 118319 words
2021-08-15 13:59:01,083 | INFO | word2vec.py:626 | prepare_vocab | Creating a fresh vocabulary
2021-08-15 13:59:01,106 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycl

In [148]:
%%time
for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

100%|██████████| 5116/5116 [00:00<00:00, 3308875.75it/s]
2021-08-15 13:59:02,172 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 5702 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T13:59:02.172869', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 13:59:02,329 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:02,330 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:02,332 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 13:59:02,333 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw wor

2021-08-15 13:59:03,083 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93202 effective words) took 0.1s, 659984 effective words/s
2021-08-15 13:59:03,083 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93202 effective words) took 0.1s, 641154 effective words/s', 'datetime': '2021-08-15T13:59:03.083503', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3394725.40it/s]
2021-08-15 13:59:03,088 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 5702 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T13:59:03.088197', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE

2021-08-15 13:59:04,006 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:04,008 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:04,010 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 13:59:04,011 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (92946 effective words) took 0.1s, 652798 effective words/s
2021-08-15 13:59:04,011 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (92946 effective words) took 0.1s, 631176 effective words/s', 'datetime': '2021-08-15T13:59:04.011847', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116

100%|██████████| 5116/5116 [00:00<00:00, 3518291.40it/s]
2021-08-15 13:59:04,841 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 5702 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T13:59:04.841199', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 13:59:04,978 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:04,981 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:04,983 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 13:59:04,984 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw wor

2021-08-15 13:59:05,760 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93089 effective words) took 0.1s, 636300 effective words/s
2021-08-15 13:59:05,761 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93089 effective words) took 0.2s, 614471 effective words/s', 'datetime': '2021-08-15T13:59:05.761541', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3605184.69it/s]
2021-08-15 13:59:05,766 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 5702 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T13:59:05.766061', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE

2021-08-15 13:59:06,666 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:06,667 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:06,668 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 13:59:06,668 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93192 effective words) took 0.1s, 685285 effective words/s
2021-08-15 13:59:06,669 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93192 effective words) took 0.1s, 664483 effective words/s', 'datetime': '2021-08-15T13:59:06.669187', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116

CPU times: user 6.59 s, sys: 709 ms, total: 7.3 s
Wall time: 4.66 s


In [149]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.dv[prefix]
        n += 1
    return vecs

In [150]:
train_vecs_dbow = get_vectors(model_ug_dbow, x_train1, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_val1, 100)

In [151]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow, train_labels)

LogisticRegression()

In [152]:
clf.score(validation_vecs_dbow, test_labels)

0.4683470105509965

In [153]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

2021-08-15 13:59:30,583 | INFO | doc2vec.py:279 | __init__ | using concatenative 500-dimensional layer1
2021-08-15 13:59:30,584 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w2,mc2,s0.001,t4)', 'datetime': '2021-08-15T13:59:30.584624', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 5116/5116 [00:00<00:00, 3501070.20it/s]
2021-08-15 13:59:30,588 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 13:59:30,589 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 13:59:30,646 | INFO | doc2vec.py:1040 | scan_vocab | collected 11564 word types and 5116 unique tags from a corpus of 5116 examples and 118319 words
2021-08-15 13:59:30,646 | INFO | word2vec.py:626 | prepare_vocab | Creatin

In [154]:
%%time
for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha
    

100%|██████████| 5116/5116 [00:00<00:00, 3370199.35it/s]
2021-08-15 13:59:42,440 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5703 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T13:59:42.440008', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 13:59:42,611 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 13:59:42,616 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:42,622 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:42,623 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 13:59:43,450 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 13:59:43,453 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:43,456 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:43,457 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 13:59:43,457 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93076 effective words) took 0.2s, 606077 effective words/s
2021-08-15 13:59:43,458 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93076 effective words) took 0.2s, 584907 effective words/s', 'datetime': '2021-08-15T13:59:43.458253', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:5

2021-08-15 13:59:44,353 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:44,356 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 13:59:44,356 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93102 effective words) took 0.2s, 491395 effective words/s
2021-08-15 13:59:44,357 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93102 effective words) took 0.2s, 458235 effective words/s', 'datetime': '2021-08-15T13:59:44.357294', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3535100.37it/s]
2021-08-15 13:59:44,361 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'

2021-08-15 13:59:45,197 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93040 effective words) took 0.2s, 598966 effective words/s
2021-08-15 13:59:45,197 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93040 effective words) took 0.2s, 578296 effective words/s', 'datetime': '2021-08-15T13:59:45.197780', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3365442.17it/s]
2021-08-15 13:59:45,202 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5703 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T13:59:45.202345', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE

2021-08-15 13:59:46,028 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93030 effective words) took 0.2s, 570009 effective words/s', 'datetime': '2021-08-15T13:59:46.028893', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3590706.03it/s]
2021-08-15 13:59:46,033 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5703 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T13:59:46.033599', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 13:59:46,171 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread fi

100%|██████████| 5116/5116 [00:00<00:00, 3547959.53it/s]
2021-08-15 13:59:46,799 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5703 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T13:59:46.799169', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 13:59:46,946 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 13:59:46,948 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 13:59:46,950 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 13:59:46,952 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 9.96 s, sys: 756 ms, total: 10.7 s
Wall time: 5.02 s


In [155]:
train_vecs_dmc = get_vectors(model_ug_dmc, x_train1, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_val1, 100)

In [156]:
clf = LogisticRegression()
clf.fit(train_vecs_dmc, train_labels)

LogisticRegression()

In [157]:
clf.score(validation_vecs_dmc,test_labels)

0.47010550996483

In [158]:
cores = multiprocessing.cpu_count()
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

2021-08-15 14:02:07,227 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w4,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:02:07.227848', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 5116/5116 [00:00<00:00, 3543272.67it/s]
2021-08-15 14:02:07,233 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:02:07,233 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:02:07,288 | INFO | doc2vec.py:1040 | scan_vocab | collected 11564 word types and 5116 unique tags from a corpus of 5116 examples and 118319 words
2021-08-15 14:02:07,289 | INFO | word2vec.py:626 | prepare_vocab | Creating a fresh vocabulary
2021-08-15 14:02:07,311 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifec

In [159]:
%%time
for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha

100%|██████████| 5116/5116 [00:00<00:00, 3403879.96it/s]
2021-08-15 14:02:16,560 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5702 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:02:16.560089', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:02:16,852 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:02:16,863 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:02:16,864 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:02:16,866 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:02:18,437 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:02:18,444 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:02:18,446 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:02:18,450 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:02:18,450 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (92950 effective words) took 0.3s, 304322 effective words/s
2021-08-15 14:02:18,451 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (92950 effective words) took 0.3s, 299059 effective words/s', 'datetime': '2021-08-15T14:02:18.451292', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:5

2021-08-15 14:02:20,185 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:02:20,187 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:02:20,187 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93009 effective words) took 0.3s, 309909 effective words/s
2021-08-15 14:02:20,188 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93009 effective words) took 0.3s, 302046 effective words/s', 'datetime': '2021-08-15T14:02:20.188818', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3281550.58it/s]
2021-08-15 14:02:20,195 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'

2021-08-15 14:02:21,896 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 118319 raw words (93107 effective words) took 0.4s, 236059 effective words/s
2021-08-15 14:02:21,896 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93107 effective words) took 0.4s, 232430 effective words/s', 'datetime': '2021-08-15T14:02:21.896870', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 1852067.95it/s]
2021-08-15 14:02:21,914 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5702 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:02:21.914624', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE

2021-08-15 14:02:23,673 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 118319 raw words (93034 effective words) took 0.3s, 284483 effective words/s', 'datetime': '2021-08-15T14:02:23.673172', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 5116/5116 [00:00<00:00, 3005751.40it/s]
2021-08-15 14:02:23,678 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5702 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:02:23.678785', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:02:23,968 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread fi

100%|██████████| 5116/5116 [00:00<00:00, 3482885.78it/s]
2021-08-15 14:02:25,436 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 5702 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:02:25.436967', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:02:25,783 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:02:25,792 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:02:25,798 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:02:25,802 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 11.7 s, sys: 2.13 s, total: 13.9 s
Wall time: 10.2 s


In [160]:
train_vecs_dmm = get_vectors(model_ug_dmm, x_train1, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_val1, 100)

In [161]:
y_train = train_labels
y_validation = test_labels
clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)

LogisticRegression()

In [162]:
clf.score(validation_vecs_dmm, y_validation)

0.49824150058616645

In [163]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [164]:
train_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_train1, 200)
validation_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_val1, 200)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [165]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc, y_train)

CPU times: user 305 ms, sys: 134 ms, total: 438 ms
Wall time: 161 ms


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [166]:
clf.score(validation_vecs_dbow_dmc, y_validation)

0.46893317702227433

In [167]:
train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train1, 200)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_val1, 200)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [168]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)

CPU times: user 218 ms, sys: 95.5 ms, total: 313 ms
Wall time: 110 ms


LogisticRegression()

In [169]:
clf.score(validation_vecs_dbow_dmm, y_validation)

0.48417350527549824

In [170]:
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser

In [171]:
tokenized_train = [t.split() for t in cleaned_tweets]

In [172]:
%%time
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)

2021-08-15 14:04:36,354 | INFO | phrases.py:585 | _learn_vocab | collecting all words and their counts
2021-08-15 14:04:36,355 | INFO | phrases.py:590 | _learn_vocab | PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-08-15 14:04:36,494 | INFO | phrases.py:611 | _learn_vocab | collected 73645 token types (unigram + bigrams) from a corpus of 118319 words and 5116 sentences
2021-08-15 14:04:36,496 | INFO | phrases.py:668 | add_vocab | merged Phrases<73645 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2021-08-15 14:04:36,538 | INFO | utils.py:448 | add_lifecycle_event | Phrases lifecycle event {'msg': 'built Phrases<73645 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000> in 0.18s', 'datetime': '2021-08-15T14:04:36.538085', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
2021-08-15 14:04:36,539 | INFO | phrases.py:777 |

CPU times: user 272 ms, sys: 29.1 ms, total: 301 ms
Wall time: 339 ms


In [191]:
def labelize_tweets_bg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(bigram[t.split()], [prefix + '_%s' % i]))
    return result

In [192]:
all_x_w2v_bg = labelize_tweets_bg(all_x, 'all')

In [193]:
cores = multiprocessing.cpu_count()
model_bg_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dbow.build_vocab([x for x in tqdm(all_x_w2v_bg)])

2021-08-15 14:07:34,126 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:07:34.125992', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 6822/6822 [00:00<00:00, 3707377.80it/s]
2021-08-15 14:07:34,132 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:07:34,132 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:07:34,199 | INFO | doc2vec.py:1040 | scan_vocab | collected 13902 word types and 5116 unique tags from a corpus of 6822 examples and 144381 words
2021-08-15 14:07:34,200 | INFO | word2vec.py:626 | prepare_vocab | Creating a fresh vocabulary
2021-08-15 14:07:34,231 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycl

In [194]:
%%time
for epoch in range(30):
    model_bg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dbow.alpha -= 0.002
    model_bg_dbow.min_alpha = model_bg_dbow.alpha

100%|██████████| 6822/6822 [00:00<00:00, 2521016.91it/s]
2021-08-15 14:07:34,393 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:07:34.393686', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:34,566 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:34,582 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:34,595 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:34,596 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:07:35,637 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:35,654 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:35,667 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:35,671 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:07:35,672 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117555 effective words) took 0.2s, 608762 effective words/s
2021-08-15 14:07:35,673 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117555 effective words) took 0.2s, 569382 effective words/s', 'datetime': '2021-08-15T14:07:35.673005', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17

2021-08-15 14:07:36,658 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:36,661 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:07:36,661 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117643 effective words) took 0.2s, 655062 effective words/s
2021-08-15 14:07:36,662 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117643 effective words) took 0.2s, 634395 effective words/s', 'datetime': '2021-08-15T14:07:36.662180', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 2731863.84it/s]
2021-08-15 14:07:36,669 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event 

2021-08-15 14:07:37,631 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117579 effective words) took 0.2s, 632144 effective words/s
2021-08-15 14:07:37,632 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117579 effective words) took 0.2s, 610990 effective words/s', 'datetime': '2021-08-15T14:07:37.632723', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3027888.03it/s]
2021-08-15 14:07:37,639 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:07:37.639596', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEA

2021-08-15 14:07:38,623 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117420 effective words) took 0.2s, 583685 effective words/s', 'datetime': '2021-08-15T14:07:38.623525', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3740332.27it/s]
2021-08-15 14:07:38,629 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:07:38.629080', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:38,791 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread f

100%|██████████| 6822/6822 [00:00<00:00, 2635007.08it/s]
2021-08-15 14:07:39,639 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:07:39.639774', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:39,792 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:39,810 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:39,823 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:39,825 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 8.55 s, sys: 895 ms, total: 9.45 s
Wall time: 6.01 s


In [195]:
train_vecs_dbow_bg = get_vectors(model_bg_dbow, x_train1, 100)
validation_vecs_dbow_bg = get_vectors(model_bg_dbow, x_val1, 100)

In [196]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_bg, y_train)

CPU times: user 46 ms, sys: 22 ms, total: 68 ms
Wall time: 27.6 ms


LogisticRegression()

In [197]:
clf.score(validation_vecs_dbow_bg, y_validation)

0.552168815943728

In [198]:
cores = multiprocessing.cpu_count()
model_bg_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dmc.build_vocab([x for x in tqdm(all_x_w2v_bg)])

2021-08-15 14:07:40,468 | INFO | doc2vec.py:279 | __init__ | using concatenative 500-dimensional layer1
2021-08-15 14:07:40,469 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w2,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:07:40.469865', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 6822/6822 [00:00<00:00, 3423081.93it/s]
2021-08-15 14:07:40,478 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:07:40,478 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:07:40,553 | INFO | doc2vec.py:1040 | scan_vocab | collected 13902 word types and 5116 unique tags from a corpus of 6822 examples and 144381 words
2021-08-15 14:07:40,553 | INFO | word2vec.py:626 | prepare_vocab | Creatin

In [199]:
%%time
for epoch in range(30):
    model_bg_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dmc.alpha -= 0.002
    model_bg_dmc.min_alpha = model_bg_dmc.alpha

100%|██████████| 6822/6822 [00:00<00:00, 3599640.44it/s]
2021-08-15 14:07:43,354 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7225 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:07:43.354201', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:43,547 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:43,564 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:43,579 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:43,582 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:07:44,764 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:44,782 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:44,798 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:44,800 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:07:44,801 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117612 effective words) took 0.2s, 502519 effective words/s
2021-08-15 14:07:44,802 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117612 effective words) took 0.2s, 490802 effective words/s', 'datetime': '2021-08-15T14:07:44.802001', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17

2021-08-15 14:07:45,982 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:45,987 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:07:45,988 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117583 effective words) took 0.2s, 533475 effective words/s
2021-08-15 14:07:45,989 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117583 effective words) took 0.2s, 517435 effective words/s', 'datetime': '2021-08-15T14:07:45.989244', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3586555.76it/s]
2021-08-15 14:07:45,994 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event 

2021-08-15 14:07:47,129 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117374 effective words) took 0.2s, 551759 effective words/s
2021-08-15 14:07:47,130 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117374 effective words) took 0.2s, 534621 effective words/s', 'datetime': '2021-08-15T14:07:47.130232', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3116943.56it/s]
2021-08-15 14:07:47,137 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7225 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:07:47.137918', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEA

2021-08-15 14:07:48,302 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117578 effective words) took 0.2s, 488134 effective words/s', 'datetime': '2021-08-15T14:07:48.301987', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3253017.50it/s]
2021-08-15 14:07:48,309 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7225 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:07:48.309710', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:48,502 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread f

100%|██████████| 6822/6822 [00:00<00:00, 3498843.47it/s]
2021-08-15 14:07:49,496 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7225 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:07:49.496885', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:49,690 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:49,709 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:49,724 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:49,728 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 13.4 s, sys: 953 ms, total: 14.3 s
Wall time: 7.07 s


In [200]:
train_vecs_dmc_bg = get_vectors(model_bg_dmc, x_train1, 100)
validation_vecs_dmc_bg = get_vectors(model_bg_dmc, x_val1, 100)

In [201]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmc_bg, y_train)

CPU times: user 79.1 ms, sys: 37.4 ms, total: 116 ms
Wall time: 47 ms


LogisticRegression()

In [202]:
clf.score(validation_vecs_dmc_bg, y_validation)

0.5134818288393904

In [203]:
cores = multiprocessing.cpu_count()
model_bg_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dmm.build_vocab([x for x in tqdm(all_x_w2v_bg)])

2021-08-15 14:07:50,506 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w4,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:07:50.506436', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 6822/6822 [00:00<00:00, 2846268.96it/s]
2021-08-15 14:07:50,514 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:07:50,515 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:07:50,578 | INFO | doc2vec.py:1040 | scan_vocab | collected 13902 word types and 5116 unique tags from a corpus of 6822 examples and 144381 words
2021-08-15 14:07:50,579 | INFO | word2vec.py:626 | prepare_vocab | Creating a fresh vocabulary
2021-08-15 14:07:50,606 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifec

In [204]:
%%time
for epoch in range(30):
    model_bg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dmm.alpha -= 0.002
    model_bg_dmm.min_alpha = model_bg_dmm.alpha

100%|██████████| 6822/6822 [00:00<00:00, 3496705.60it/s]
2021-08-15 14:07:55,204 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:07:55.204459', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:07:55,523 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:55,555 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:55,582 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:55,591 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:07:57,563 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:07:57,588 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:07:57,610 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:57,616 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:07:57,616 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117759 effective words) took 0.4s, 301764 effective words/s
2021-08-15 14:07:57,618 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117759 effective words) took 0.4s, 294414 effective words/s', 'datetime': '2021-08-15T14:07:57.618029', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17

2021-08-15 14:07:59,551 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:07:59,556 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:07:59,557 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117334 effective words) took 0.4s, 316882 effective words/s
2021-08-15 14:07:59,557 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117334 effective words) took 0.4s, 309682 effective words/s', 'datetime': '2021-08-15T14:07:59.557791', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3285891.35it/s]
2021-08-15 14:07:59,564 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event 

2021-08-15 14:08:01,572 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 144381 raw words (117569 effective words) took 0.4s, 323212 effective words/s
2021-08-15 14:08:01,573 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117569 effective words) took 0.4s, 317970 effective words/s', 'datetime': '2021-08-15T14:08:01.573259', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3607810.10it/s]
2021-08-15 14:08:01,579 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:08:01.579001', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEA

2021-08-15 14:08:03,485 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 144381 raw words (117619 effective words) took 0.4s, 325855 effective words/s', 'datetime': '2021-08-15T14:08:03.485052', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3611908.85it/s]
2021-08-15 14:08:03,490 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:08:03.490673', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:08:03,808 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread f

100%|██████████| 6822/6822 [00:00<00:00, 3350924.22it/s]
2021-08-15 14:08:05,360 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7224 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:08:05.360196', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:08:05,702 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:08:05,731 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:08:05,748 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:08:05,754 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 14.7 s, sys: 2.76 s, total: 17.5 s
Wall time: 11.8 s


In [205]:
train_vecs_dmm_bg = get_vectors(model_bg_dmm, x_train1, 100)
validation_vecs_dmm_bg = get_vectors(model_bg_dmm, x_val1, 100)

In [206]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmm_bg, y_train)

CPU times: user 55.1 ms, sys: 35.5 ms, total: 90.6 ms
Wall time: 38.8 ms


LogisticRegression()

In [207]:
clf.score(validation_vecs_dmm_bg, y_validation)

0.5193434935521688

In [208]:
train_vecs_dbow_dmc_bg = get_concat_vectors(model_bg_dbow,model_bg_dmc, x_train1, 200)
validation_vecs_dbow_dmc_bg = get_concat_vectors(model_bg_dbow,model_bg_dmc, x_val1, 200)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [209]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc_bg, y_train)

CPU times: user 315 ms, sys: 142 ms, total: 457 ms
Wall time: 163 ms


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [210]:
clf.score(validation_vecs_dbow_dmc_bg, y_validation)

0.5398593200468933

In [211]:
train_vecs_dbow_dmm_bg = get_concat_vectors(model_bg_dbow,model_bg_dmm, x_train1, 200)
validation_vecs_dbow_dmm_bg = get_concat_vectors(model_bg_dbow,model_bg_dmm, x_val1, 200)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [212]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm_bg, y_train)

CPU times: user 190 ms, sys: 87.8 ms, total: 278 ms
Wall time: 96.1 ms


LogisticRegression()

In [213]:
clf.score(validation_vecs_dbow_dmm_bg, y_validation)

0.5451348182883939

In [214]:
%%time
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)

2021-08-15 14:09:29,808 | INFO | phrases.py:585 | _learn_vocab | collecting all words and their counts
2021-08-15 14:09:29,809 | INFO | phrases.py:590 | _learn_vocab | PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-08-15 14:09:30,045 | INFO | phrases.py:611 | _learn_vocab | collected 76187 token types (unigram + bigrams) from a corpus of 108386 words and 5116 sentences
2021-08-15 14:09:30,046 | INFO | phrases.py:668 | add_vocab | merged Phrases<76187 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2021-08-15 14:09:30,046 | INFO | utils.py:448 | add_lifecycle_event | Phrases lifecycle event {'msg': 'built Phrases<76187 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000> in 0.24s', 'datetime': '2021-08-15T14:09:30.046743', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
2021-08-15 14:09:30,047 | INFO | phrases.py:777 |

CPU times: user 373 ms, sys: 9.87 ms, total: 383 ms
Wall time: 392 ms


In [215]:
def labelize_tweets_tg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result

In [216]:
all_x_w2v_tg = labelize_tweets_tg(all_x, 'all')

In [217]:
model_tg_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dbow.build_vocab([x for x in tqdm(all_x_w2v_tg)])

2021-08-15 14:09:55,054 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:09:55.054099', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 6822/6822 [00:00<00:00, 2238931.29it/s]
2021-08-15 14:09:55,059 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:09:55,060 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:09:55,122 | INFO | doc2vec.py:1040 | scan_vocab | collected 14016 word types and 5116 unique tags from a corpus of 6822 examples and 139352 words
2021-08-15 14:09:55,123 | INFO | word2vec.py:626 | prepare_vocab | Creating a fresh vocabulary
2021-08-15 14:09:55,151 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycl

In [218]:
%%time
for epoch in range(30):
    model_tg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dbow.alpha -= 0.002
    model_tg_dbow.min_alpha = model_tg_dbow.alpha

100%|██████████| 6822/6822 [00:00<00:00, 2997437.87it/s]
2021-08-15 14:10:03,560 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:10:03.560687', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:10:03,720 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:10:03,724 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:10:03,746 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:03,747 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:10:04,833 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:10:04,836 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:10:04,858 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:04,860 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:10:04,860 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114107 effective words) took 0.2s, 520254 effective words/s
2021-08-15 14:10:04,861 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114107 effective words) took 0.2s, 507094 effective words/s', 'datetime': '2021-08-15T14:10:04.861100', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17

2021-08-15 14:10:05,906 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:05,907 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:10:05,908 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114119 effective words) took 0.2s, 618168 effective words/s
2021-08-15 14:10:05,909 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114119 effective words) took 0.2s, 596340 effective words/s', 'datetime': '2021-08-15T14:10:05.908983', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3516473.13it/s]
2021-08-15 14:10:05,914 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event 

2021-08-15 14:10:06,952 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114190 effective words) took 0.2s, 559439 effective words/s
2021-08-15 14:10:06,953 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114190 effective words) took 0.2s, 543299 effective words/s', 'datetime': '2021-08-15T14:10:06.953471', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3015761.16it/s]
2021-08-15 14:10:06,960 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:10:06.960649', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEA

2021-08-15 14:10:08,042 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114130 effective words) took 0.2s, 542566 effective words/s', 'datetime': '2021-08-15T14:10:08.042896', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 2845136.91it/s]
2021-08-15 14:10:08,049 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:10:08.049082', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:10:08,219 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread f

100%|██████████| 6822/6822 [00:00<00:00, 3529050.55it/s]
2021-08-15 14:10:09,069 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5', 'datetime': '2021-08-15T14:10:09.069073', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:10:09,233 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:10:09,235 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:10:09,261 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:09,262 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 8.69 s, sys: 891 ms, total: 9.58 s
Wall time: 6.28 s


In [219]:
train_vecs_dbow_tg = get_vectors(model_tg_dbow, x_train1, 100)
validation_vecs_dbow_tg = get_vectors(model_tg_dbow, x_val1, 100)

In [220]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_tg, y_train)

CPU times: user 33.3 ms, sys: 20 ms, total: 53.4 ms
Wall time: 19.6 ms


LogisticRegression()

In [221]:
clf.score(validation_vecs_dbow_tg, y_validation)

0.5539273153575616

In [222]:
cores = multiprocessing.cpu_count()
model_tg_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmc.build_vocab([x for x in tqdm(all_x_w2v_tg)])

2021-08-15 14:10:46,981 | INFO | doc2vec.py:279 | __init__ | using concatenative 500-dimensional layer1
2021-08-15 14:10:46,982 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w2,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:10:46.982437', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 6822/6822 [00:00<00:00, 3523401.29it/s]
2021-08-15 14:10:46,986 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:10:46,987 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:10:47,050 | INFO | doc2vec.py:1040 | scan_vocab | collected 14016 word types and 5116 unique tags from a corpus of 6822 examples and 139352 words
2021-08-15 14:10:47,050 | INFO | word2vec.py:626 | prepare_vocab | Creatin

In [223]:
%%time
for epoch in range(30):
    model_tg_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dmc.alpha -= 0.002
    model_tg_dmc.min_alpha = model_tg_dmc.alpha

100%|██████████| 6822/6822 [00:00<00:00, 3477159.06it/s]
2021-08-15 14:10:56,214 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7310 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:10:56.214526', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:10:56,395 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:10:56,397 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:10:56,426 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:56,427 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:10:57,713 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:10:57,713 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:10:57,717 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:57,742 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:10:57,742 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114122 effective words) took 0.3s, 403937 effective words/s
2021-08-15 14:10:57,743 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114122 effective words) took 0.3s, 392423 effective words/s', 'datetime': '2021-08-15T14:10:57.743148', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17

2021-08-15 14:10:58,859 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:10:58,859 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:10:58,860 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114117 effective words) took 0.2s, 542272 effective words/s
2021-08-15 14:10:58,861 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114117 effective words) took 0.2s, 526054 effective words/s', 'datetime': '2021-08-15T14:10:58.861899', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3471674.58it/s]
2021-08-15 14:10:58,868 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event 

2021-08-15 14:10:59,997 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114054 effective words) took 0.2s, 511297 effective words/s
2021-08-15 14:10:59,998 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114054 effective words) took 0.2s, 493105 effective words/s', 'datetime': '2021-08-15T14:10:59.998550', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 2738662.13it/s]
2021-08-15 14:11:00,007 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7310 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:11:00.007090', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEA

2021-08-15 14:11:01,220 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114104 effective words) took 0.3s, 432011 effective words/s', 'datetime': '2021-08-15T14:11:01.220761', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 2816847.99it/s]
2021-08-15 14:11:01,227 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7310 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:11:01.227209', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:11:01,421 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread f

100%|██████████| 6822/6822 [00:00<00:00, 3733986.94it/s]
2021-08-15 14:11:02,343 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7310 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-08-15T14:11:02.343668', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:11:02,506 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:11:02,508 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:11:02,538 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:11:02,541 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 13.1 s, sys: 914 ms, total: 14 s
Wall time: 6.98 s


In [224]:
train_vecs_dmc_tg = get_vectors(model_tg_dmc, x_train1, 100)
validation_vecs_dmc_tg = get_vectors(model_tg_dmc, x_val1, 100)

In [225]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmc_tg, y_train)

CPU times: user 80.6 ms, sys: 50 ms, total: 131 ms
Wall time: 46.4 ms


LogisticRegression()

In [226]:
clf.score(validation_vecs_dmc_tg, y_validation)

0.518757327080891

In [227]:
cores = multiprocessing.cpu_count()
model_tg_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmm.build_vocab([x for x in tqdm(all_x_w2v_tg)])

2021-08-15 14:11:39,563 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w4,mc2,s0.001,t4)', 'datetime': '2021-08-15T14:11:39.563175', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 6822/6822 [00:00<00:00, 3505702.27it/s]
2021-08-15 14:11:39,567 | INFO | doc2vec.py:1032 | scan_vocab | collecting all words and their counts
2021-08-15 14:11:39,568 | INFO | doc2vec.py:951 | _scan_vocab | PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-08-15 14:11:39,629 | INFO | doc2vec.py:1040 | scan_vocab | collected 14016 word types and 5116 unique tags from a corpus of 6822 examples and 139352 words
2021-08-15 14:11:39,630 | INFO | word2vec.py:626 | prepare_vocab | Creating a fresh vocabulary
2021-08-15 14:11:39,656 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifec

In [228]:
%%time
for epoch in range(30):
    model_tg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dmm.alpha -= 0.002
    model_tg_dmc.min_alpha = model_tg_dmc.alpha

100%|██████████| 6822/6822 [00:00<00:00, 3542156.71it/s]
2021-08-15 14:11:48,468 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:11:48.468731', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:11:48,808 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:11:48,812 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:11:48,849 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:11:48,851 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

2021-08-15 14:11:50,776 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:11:50,779 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:11:50,823 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:11:50,827 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:11:50,828 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114075 effective words) took 0.4s, 299733 effective words/s
2021-08-15 14:11:50,828 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114075 effective words) took 0.4s, 289031 effective words/s', 'datetime': '2021-08-15T14:11:50.828701', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17

2021-08-15 14:11:52,846 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:11:52,850 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 0 more threads
2021-08-15 14:11:52,851 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (113999 effective words) took 0.4s, 286759 effective words/s
2021-08-15 14:11:52,851 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (113999 effective words) took 0.4s, 282509 effective words/s', 'datetime': '2021-08-15T14:11:52.851669', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3642716.98it/s]
2021-08-15 14:11:52,857 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event 

2021-08-15 14:11:54,738 | INFO | word2vec.py:1624 | _log_epoch_end | EPOCH - 1 : training on 139352 raw words (114192 effective words) took 0.4s, 309088 effective words/s
2021-08-15 14:11:54,739 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114192 effective words) took 0.4s, 302423 effective words/s', 'datetime': '2021-08-15T14:11:54.739468', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 3522100.18it/s]
2021-08-15 14:11:54,745 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:11:54.745223', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEA

2021-08-15 14:11:56,834 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training on 139352 raw words (114375 effective words) took 0.5s, 242826 effective words/s', 'datetime': '2021-08-15T14:11:56.834645', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
100%|██████████| 6822/6822 [00:00<00:00, 2651611.70it/s]
2021-08-15 14:11:56,845 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:11:56.845647', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:11:57,186 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread f

100%|██████████| 6822/6822 [00:00<00:00, 3670285.00it/s]
2021-08-15 14:11:58,873 | INFO | utils.py:448 | add_lifecycle_event | Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 7309 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4', 'datetime': '2021-08-15T14:11:58.873232', 'gensim': '4.0.1', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
2021-08-15 14:11:59,190 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 3 more threads
2021-08-15 14:11:59,195 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 2 more threads
2021-08-15 14:11:59,239 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting finish of 1 more threads
2021-08-15 14:11:59,241 | INFO | word2vec.py:1281 | _log_epoch_progress | worker thread finished; awaiting 

CPU times: user 14.5 s, sys: 2.67 s, total: 17.2 s
Wall time: 11.9 s


In [229]:
train_vecs_dmm_tg = get_vectors(model_tg_dmm, x_train1, 100)
validation_vecs_dmm_tg = get_vectors(model_tg_dmm, x_val1, 100)

In [230]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmm_tg, y_train)
clf.score(validation_vecs_dmm_tg, y_validation)

CPU times: user 42.3 ms, sys: 27.6 ms, total: 69.8 ms
Wall time: 25.5 ms


0.5181711606096131

In [232]:
train_vecs_dbow_dmc_tg = get_concat_vectors(model_tg_dbow,model_tg_dmc, x_train1, 200)
validation_vecs_dbow_dmc_tg = get_concat_vectors(model_tg_dbow,model_tg_dmc, x_val1, 200)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [233]:

clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc_tg, y_train)
clf.score(validation_vecs_dbow_dmc_tg, y_validation)

0.5486518171160609

In [234]:
train_vecs_dbow_dmm_tg = get_concat_vectors(model_tg_dbow,model_tg_dmm, x_train1, 200)
validation_vecs_dbow_dmm_tg = get_concat_vectors(model_tg_dbow,model_tg_dmm, x_val1, 200)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [235]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm_tg, y_train)
clf.score(validation_vecs_dbow_dmm_tg, y_validation)

0.5404454865181711

In [236]:
train_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_train1, 200)
validation_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_val1, 200)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  


In [237]:
clf = LogisticRegression()
clf.fit(train_vecs_ugdbow_tgdmm, y_train)

LogisticRegression()

In [238]:
clf.score(validation_vecs_ugdbow_tgdmm, y_validation)

0.488862837045721